📦 Cellule 1 : Imports et Configuration

In [3]:
# ========================================
# IMPORTS
# ========================================
import pandas as pd
import pyodbc
from dotenv import load_dotenv
import os
from tqdm import tqdm
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

print("=" * 70)
print("📊 ETL - CHARGEMENT DE DIM_ENTREPRISE DEPUIS CSV LOCAL")
print("=" * 70)

# ========================================
# CHARGEMENT DES VARIABLES D'ENVIRONNEMENT
# ========================================
load_dotenv()

# Configuration DWH - UTILISER LES BONS NOMS DE VARIABLES
DB_SERVER_DWH = os.getenv('DB_SERVER_DWH-student')
DB_DATABASE_DWH = os.getenv('DB_DATABASE_DWH')
DB_USERNAME_DWH = os.getenv('DB_USERNAME_DWH')
DB_PASSWORD_DWH = os.getenv('DB_PASSWORD_DWH')

# Chemin du fichier CSV local
CSV_FILE_PATH = "siren_naf_45.csv"

print("\n✅ Variables d'environnement chargées")
print(f"   • DWH Server : {DB_SERVER_DWH}")
print(f"   • DWH Database : {DB_DATABASE_DWH}")
print(f"   • DWH Username : {DB_USERNAME_DWH}")
print(f"   • CSV File : {CSV_FILE_PATH}")

# Vérifier que les variables sont bien chargées
if not DB_SERVER_DWH or not DB_USERNAME_DWH or not DB_PASSWORD_DWH:
    print("\n❌ ERREUR : Variables d'environnement manquantes!")
    print("   Vérifie ton fichier .env")
    raise ValueError("Variables d'environnement manquantes")

📊 ETL - CHARGEMENT DE DIM_ENTREPRISE DEPUIS CSV LOCAL

✅ Variables d'environnement chargées
   • DWH Server : carter-cash-serveur-student.database.windows.net
   • DWH Database : DWH_E5_projet_AUTO
   • DWH Username : carter-cash-serveur
   • CSV File : siren_naf_45.csv


📦 Cellule 2 : Connexion au DWH

In [4]:
print("\n" + "=" * 70)
print("🔗 CONNEXION AU DATA WAREHOUSE")
print("=" * 70)

try:
    # Méthode qui fonctionne (basée sur ton code qui marche)
    driver = '{ODBC Driver 17 for SQL Server}'
    
    connection_string_dwh = (
        'DRIVER=' + driver + 
        ';SERVER=' + DB_SERVER_DWH + 
        ';PORT=1433;DATABASE=' + DB_DATABASE_DWH + 
        ';UID=' + DB_USERNAME_DWH + 
        ';PWD=' + DB_PASSWORD_DWH
    )
    
    cnxn_dwh = pyodbc.connect(connection_string_dwh)
    cursor_dwh = cnxn_dwh.cursor()
    
    print(f"✅ Connexion réussie au DWH")
    print(f"   • Serveur : {DB_SERVER_DWH}")
    print(f"   • Base de données : {DB_DATABASE_DWH}")
    print(f"   • Utilisateur : {DB_USERNAME_DWH}")
    
except Exception as e:
    print(f"❌ Erreur de connexion au DWH : {e}")
    print(f"\n🔍 DEBUG - Valeurs des variables :")
    print(f"   • Server : {DB_SERVER_DWH}")
    print(f"   • Database : {DB_DATABASE_DWH}")
    print(f"   • Username : {DB_USERNAME_DWH}")
    print(f"   • Password : {'*' * len(DB_PASSWORD_DWH) if DB_PASSWORD_DWH else 'None'}")
    raise


🔗 CONNEXION AU DATA WAREHOUSE
✅ Connexion réussie au DWH
   • Serveur : carter-cash-serveur-student.database.windows.net
   • Base de données : DWH_E5_projet_AUTO
   • Utilisateur : carter-cash-serveur


📦 Cellule 3 : Lecture du fichier CSV


In [5]:
print("\n" + "=" * 70)
print("📥 LECTURE DU FICHIER CSV")
print("=" * 70)

try:
    # Vérifier que le fichier existe
    if not os.path.exists(CSV_FILE_PATH):
        raise FileNotFoundError(f"Le fichier {CSV_FILE_PATH} n'existe pas")
    
    print(f"\n⏳ Lecture du fichier : {CSV_FILE_PATH}...")
    
    # Lire le CSV avec séparateur ";"
    df_entreprises = pd.read_csv(
        CSV_FILE_PATH,
        sep=';',
        dtype=str,  # Tout en string pour l'instant
        encoding='utf-8'
    )
    
    print(f"✅ Fichier chargé : {len(df_entreprises)} lignes × {len(df_entreprises.columns)} colonnes")
    
    print(f"\n📊 COLONNES DU CSV :")
    print(list(df_entreprises.columns))
    
    print(f"\n📊 APERÇU DES PREMIÈRES LIGNES :")
    print(df_entreprises.head(10))
    
    print(f"\n📊 STATISTIQUES :")
    print(f"   • Nombre total d'établissements : {len(df_entreprises)}")
    print(f"   • Nombre de SIREN uniques : {df_entreprises['siren'].nunique()}")
    print(f"   • Nombre de SIRET uniques : {df_entreprises['siret'].nunique()}")
    
except Exception as e:
    print(f"❌ Erreur lors de la lecture du CSV : {e}")
    raise



📥 LECTURE DU FICHIER CSV

⏳ Lecture du fichier : siren_naf_45.csv...
✅ Fichier chargé : 19028 lignes × 17 colonnes

📊 COLONNES DU CSV :
['siren', 'siret', 'dateCreationEtablissement', 'trancheEffectifsEtablissement', 'activitePrincipaleRegistreMetiersEtablissement', 'dateDernierTraitementEtablissement', 'complementAdresseEtablissement', 'codePostalEtablissement', 'codeCommuneEtablissement', 'complementAdresse2Etablissement', 'dateDebut', 'etatAdministratifEtablissement', 'enseigne1Etablissement', 'enseigne2Etablissement', 'enseigne3Etablissement', 'denominationUsuelleEtablissement', 'activitePrincipaleEtablissement']

📊 APERÇU DES PREMIÈRES LIGNES :
      siren          siret dateCreationEtablissement  \
0   7211253   721125300021                1996-04-01   
1   7320161   732016100024                1999-03-31   
2   7320187   732018700029                1900-01-01   
3   7320187   732018700052                2018-06-01   
4   7320294   732029400023                1900-01-01   
5  25

📦 Cellule 4 : Transformation et Mapping des données


In [6]:
print("\n" + "=" * 70)
print("🔄 TRANSFORMATION ET MAPPING DES DONNÉES")
print("=" * 70)

# ========================================
# FONCTION DE FALLBACK POUR RÉCUPÉRER LA PREMIÈRE VALEUR NON VIDE
# ========================================
def get_first_non_empty(*values):
    """Retourne la première valeur non vide/non NaN parmi plusieurs colonnes"""
    for val in values:
        if pd.notna(val) and str(val).strip() != '' and str(val).strip().lower() != 'nan':
            return str(val).strip()
    return ''

# ========================================
# FONCTION DE CONVERSION DE DATE
# ========================================
def convert_to_date(date_str):
    """Convertit une chaîne de date en format DATE SQL"""
    if pd.isna(date_str) or str(date_str).strip() == '' or str(date_str).strip().lower() == 'nan':
        return None
    
    try:
        # Format attendu : YYYY-MM-DD ou YYYY-MM-DDTHH:MM:SS
        date_str = str(date_str).strip()
        
        # Si format avec heure, on garde juste la date
        if 'T' in date_str:
            date_str = date_str.split('T')[0]
        
        # Convertir en objet date
        date_obj = pd.to_datetime(date_str, format='%Y-%m-%d', errors='coerce')
        
        if pd.isna(date_obj):
            return None
        
        return date_obj.date()
    except:
        return None

# ========================================
# CRÉER LE DATAFRAME POUR DIM_ENTREPRISE
# ========================================
print("\n⏳ Application des règles de mapping...")

df_dim_entreprise = pd.DataFrame()

# SIREN (direct)
df_dim_entreprise['SIREN'] = df_entreprises['siren'].fillna('').astype(str).str.strip()

# SIRET (direct)
df_dim_entreprise['SIRET'] = df_entreprises['siret'].fillna('').astype(str).str.strip()

# NAF (fallback: activitePrincipaleRegistreMetiersEtablissement → activitePrincipaleEtablissement)
df_dim_entreprise['NAF'] = df_entreprises.apply(
    lambda row: get_first_non_empty(
        row['activitePrincipaleRegistreMetiersEtablissement'],
        row['activitePrincipaleEtablissement']
    ),
    axis=1
)

# Denomination (fallback: denominationUsuelleEtablissement → enseigne1 → enseigne2 → enseigne3)
df_dim_entreprise['Denomination'] = df_entreprises.apply(
    lambda row: get_first_non_empty(
        row['denominationUsuelleEtablissement'],
        row['enseigne1Etablissement'],
        row['enseigne2Etablissement'],
        row['enseigne3Etablissement']
    ),
    axis=1
)

# Adresse_Complete (fallback: complementAdresseEtablissement → complementAdresse2Etablissement)
df_dim_entreprise['Adresse_Complete'] = df_entreprises.apply(
    lambda row: get_first_non_empty(
        row['complementAdresseEtablissement'],
        row['complementAdresse2Etablissement']
    ),
    axis=1
)

# Code_Commune (fallback: codeCommuneEtablissement → codePostalEtablissement)
df_dim_entreprise['Code_Commune'] = df_entreprises.apply(
    lambda row: get_first_non_empty(
        row['codeCommuneEtablissement'],
        row['codePostalEtablissement']
    ),
    axis=1
)

# Tranche_Effectifs (direct)
df_dim_entreprise['Tranche_Effectifs'] = df_entreprises['trancheEffectifsEtablissement'].fillna('').astype(str).str.strip()

# Date_Creation (fallback: dateCreationEtablissement → dateDebut)
df_dim_entreprise['Date_Creation'] = df_entreprises.apply(
    lambda row: convert_to_date(get_first_non_empty(
        row['dateCreationEtablissement'],
        row['dateDebut']
    )),
    axis=1
)

# Etat_Administratif (direct)
df_dim_entreprise['Etat_Administratif'] = df_entreprises['etatAdministratifEtablissement'].fillna('').astype(str).str.strip()

# Date_Debut_Validite (direct)
df_dim_entreprise['Date_Debut_Validite'] = df_entreprises['dateDebut'].apply(convert_to_date)

# Date_Fin_Validite (direct)
df_dim_entreprise['Date_Fin_Validite'] = df_entreprises['dateDernierTraitementEtablissement'].apply(convert_to_date)

# Flag_Actuel (basé sur Etat_Administratif = 'A' pour Actif)
df_dim_entreprise['Flag_Actuel'] = df_dim_entreprise['Etat_Administratif'].apply(
    lambda x: 1 if x == 'A' else 0
)

print(f"✅ Mapping terminé")

# ========================================
# NETTOYAGE ET VALIDATION
# ========================================
print("\n⏳ Nettoyage et validation des données...")

# Supprimer les lignes sans SIRET (obligatoire)
nb_avant = len(df_dim_entreprise)
df_dim_entreprise = df_dim_entreprise[df_dim_entreprise['SIRET'] != '']
nb_apres = len(df_dim_entreprise)

if nb_avant != nb_apres:
    print(f"⚠️  {nb_avant - nb_apres} lignes supprimées (SIRET vide)")

# Supprimer les doublons sur SIRET
nb_doublons = df_dim_entreprise['SIRET'].duplicated().sum()
if nb_doublons > 0:
    print(f"⚠️  {nb_doublons} doublons trouvés sur SIRET")
    df_dim_entreprise = df_dim_entreprise.drop_duplicates(subset=['SIRET'], keep='first')

print(f"✅ {len(df_dim_entreprise)} entreprises uniques prêtes pour l'insertion")

# ========================================
# AFFICHER DES EXEMPLES
# ========================================
print(f"\n📊 APERÇU DES DONNÉES TRANSFORMÉES :")
print("=" * 70)
print(df_dim_entreprise.head(10))

print(f"\n📊 STATISTIQUES :")
print(f"   • Entreprises avec dénomination : {(df_dim_entreprise['Denomination'] != '').sum()}")
print(f"   • Entreprises avec adresse : {(df_dim_entreprise['Adresse_Complete'] != '').sum()}")
print(f"   • Entreprises actives (Flag_Actuel=1) : {df_dim_entreprise['Flag_Actuel'].sum()}")
print(f"   • Entreprises par état administratif :")

for etat in df_dim_entreprise['Etat_Administratif'].value_counts().items():
    print(f"      - {etat[0]} : {etat[1]}")


🔄 TRANSFORMATION ET MAPPING DES DONNÉES

⏳ Application des règles de mapping...
✅ Mapping terminé

⏳ Nettoyage et validation des données...
✅ 19028 entreprises uniques prêtes pour l'insertion

📊 APERÇU DES DONNÉES TRANSFORMÉES :
      SIREN          SIRET     NAF Denomination              Adresse_Complete  \
0   7211253   721125300021  4520BC                                              
1   7320161   732016100024  4520AB                                              
2   7320187   732018700029  45.31Z                                              
3   7320187   732018700052  45.31Z                                              
4   7320294   732029400023  45.11Z                                              
5  25480401  2548040101053  45.31Z                                         2-   
6  25480401  2548040101087  45.31Z     BARRAULT                                 
7  25480401  2548040101095  45.31Z     BARRAULT  ZAC LES PORTES DE LA VILETTE   
8  25480401  2548040101129  45.31Z     BA

📦 Cellule 5 : Chargement dans DIM_ENTREPRISE


In [7]:
print("\n" + "=" * 70)
print("📥 ETL - CHARGEMENT DE DIM_ENTREPRISE")
print("=" * 70)

# ========================================
# CHARGER LES SIRET EXISTANTS
# ========================================
print("\n⏳ Chargement des SIRET déjà présents dans le DWH...")
cursor_dwh.execute("SELECT SIRET FROM DIM_ENTREPRISE")
siret_existants = set(row[0] for row in cursor_dwh.fetchall())
print(f"✅ {len(siret_existants)} SIRET déjà présents dans DIM_ENTREPRISE")

# ========================================
# FILTRER LES NOUVELLES ENTREPRISES
# ========================================
df_entreprises_nouvelles = df_dim_entreprise[~df_dim_entreprise['SIRET'].isin(siret_existants)]
skip_count = len(df_dim_entreprise) - len(df_entreprises_nouvelles)

print(f"✅ {len(df_entreprises_nouvelles)} nouvelles entreprises à insérer")
print(f"⚠️  {skip_count} entreprises déjà existantes (ignorées)")

# ========================================
# INSERTION PAR BATCH
# ========================================
if len(df_entreprises_nouvelles) > 0:
    print("\n⏳ Insertion des nouvelles entreprises...")
    
    insert_query = """
    INSERT INTO DIM_ENTREPRISE (
        SIREN,
        SIRET,
        NAF,
        Denomination,
        Adresse_Complete,
        Code_Commune,
        Tranche_Effectifs,
        Date_Creation,
        Etat_Administratif,
        Date_Debut_Validite,
        Date_Fin_Validite,
        Flag_Actuel
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    
    batch_size = 500
    total_insert = 0
    
    # Préparer les tuples pour l'insertion
    tuples = []
    for _, row in df_entreprises_nouvelles.iterrows():
        tuples.append((
            row['SIREN'] if row['SIREN'] != '' else None,
            row['SIRET'] if row['SIRET'] != '' else None,
            row['NAF'] if row['NAF'] != '' else None,
            row['Denomination'] if row['Denomination'] != '' else None,
            row['Adresse_Complete'] if row['Adresse_Complete'] != '' else None,
            row['Code_Commune'] if row['Code_Commune'] != '' else None,
            row['Tranche_Effectifs'] if row['Tranche_Effectifs'] != '' else None,
            row['Date_Creation'],
            row['Etat_Administratif'] if row['Etat_Administratif'] != '' else None,
            row['Date_Debut_Validite'],
            row['Date_Fin_Validite'],
            row['Flag_Actuel']
        ))
    
    # Insertion par batch
    for i in tqdm(range(0, len(tuples), batch_size), desc="Insertion entreprises", unit="batch"):
        batch = tuples[i:i+batch_size]
        try:
            cursor_dwh.executemany(insert_query, batch)
            cnxn_dwh.commit()
            total_insert += len(batch)
        except Exception as e:
            print(f"\n⚠️  Erreur batch {i//batch_size+1}: {e}")
            # Insérer ligne par ligne en cas d'erreur
            for row_data in batch:
                try:
                    cursor_dwh.execute(insert_query, row_data)
                    cnxn_dwh.commit()
                    total_insert += 1
                except Exception as e_detail:
                    print(f"   ❌ Erreur : {e_detail}")
                    print(f"   SIRET : {row_data[1]}")
    
    print(f"\n✅ Insertion terminée : {total_insert} entreprises insérées")
else:
    print("\n✅ Aucune nouvelle entreprise à insérer")
    total_insert = 0


📥 ETL - CHARGEMENT DE DIM_ENTREPRISE

⏳ Chargement des SIRET déjà présents dans le DWH...
✅ 0 SIRET déjà présents dans DIM_ENTREPRISE
✅ 19028 nouvelles entreprises à insérer
⚠️  0 entreprises déjà existantes (ignorées)

⏳ Insertion des nouvelles entreprises...


Insertion entreprises: 100%|██████████| 39/39 [03:09<00:00,  4.86s/batch]


✅ Insertion terminée : 19028 entreprises insérées


In [8]:
print("\n" + "=" * 70)
print("🔍 VÉRIFICATION FINALE")
print("=" * 70)

# ========================================
# STATISTIQUES GLOBALES
# ========================================
cursor_dwh.execute("SELECT COUNT(*) FROM DIM_ENTREPRISE")
total_final = cursor_dwh.fetchone()[0]
print(f"\n📊 Nombre total d'entreprises dans DIM_ENTREPRISE : {total_final}")

# Répartition par état administratif
print("\n📈 RÉPARTITION PAR ÉTAT ADMINISTRATIF :")
cursor_dwh.execute("""
SELECT 
    Etat_Administratif,
    COUNT(*) as Nb_Entreprises
FROM DIM_ENTREPRISE
GROUP BY Etat_Administratif
ORDER BY Nb_Entreprises DESC
""")

for row in cursor_dwh.fetchall():
    etat = row[0] if row[0] else 'Non renseigné'
    print(f"   • {etat} : {row[1]} entreprises")

# Répartition par tranche d'effectifs
print("\n📈 RÉPARTITION PAR TRANCHE D'EFFECTIFS :")
cursor_dwh.execute


🔍 VÉRIFICATION FINALE

📊 Nombre total d'entreprises dans DIM_ENTREPRISE : 19028

📈 RÉPARTITION PAR ÉTAT ADMINISTRATIF :
   • A : 19028 entreprises

📈 RÉPARTITION PAR TRANCHE D'EFFECTIFS :


<function Cursor.execute>